In [ ]:
!git clone https://github.com/ARIM-Academy/Advanced_Tutorial_6.git
%cd Advanced_Tutorial_6

## Train your Unet with membrane data
membrane data is in folder membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### ライブラリインポート

In [ ]:
#汎用ライブラリ
import matplotlib.pyplot as plt
from PIL import Image

# モジュール
from model import *
from data import *

## 2. データ合成

In [ ]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

myGene = trainGenerator(20,
                        'data/membrane/train',
                        'image',
                        'label',
                        data_gen_args,
                        save_to_dir = "data/membrane/train/aug"
                       )

### 合成データの作成と可視化

In [ ]:
#you will see 60 transformed images and their masks in data/membrane/train/aug
num_batch = 3
for i,batch in enumerate(myGene):
    if(i >= num_batch):
        break

## npyファイルの作成

In [ ]:
image_arr,mask_arr = geneTrainNpy("data/membrane/train/aug/","data/membrane/train/aug/")
np.save("data/image_arr.npy",image_arr)
np.save("data/mask_arr.npy",mask_arr)

## 3.セマンティックセグメンテーション（U-Net）

In [ ]:
model = unet()

# ModelCheckpoint の設定（損失値に基づいて最良モデルを保存）
model_checkpoint = ModelCheckpoint('unet_membrane.keras', 
                                   monitor='loss', 
                                   verbose=1, 
                                   save_best_only=True)

Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
Epoch 1/5
   8/2000 ━━━━━━━━━━━━━━━━━━━━ 1:26:33 3s/step - accuracy: 0.6150 - loss: 0.6950

### モデル最適化

In [ ]:
history = model.fit(imgs_train,
                    imgs_mask_train,
                    batch_size = 100,
                    epochs = 50,
                    verbose = 1,
                    validation_split = 0.2, 
                    shuffle = True,
                    callbacks=[model_checkpoint]
                   )

In [ ]:
#history = model.fit(myGene,
#                    steps_per_epoch=300,
#                    epochs=10,
#                    verbose = 1,
#                    validation_split = 0.2, 
#                    shuffle = True,
#                    callbacks=[model_checkpoint]
#                    )

### 結果出力

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

### テストデータによる検証

In [ ]:
testGene = testGenerator("data/membrane/test")

model = unet()
model.load_weights("unet_membrane.keras")
results = model.predict(testGene,30,verbose=1)

### 結果の保存

In [ ]:
saveResult("data/membrane/test/test_results",results)